In [4]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Reshape
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.preprocessing import MinMaxScaler

# 1️⃣ Wczytanie danych
X = np.load("X_weather.npy")  # shape: (samples, 14, 10) - wszystkie cechy
y = np.load("y_weather.npy")  # shape: (samples, 7, 10)

# 🔹 Wybór tylko tavg jako wyjście
y = y[:, :, 0].reshape(-1, 7, 1)  # shape: (samples, 7, 1)

# 2️⃣ Normalizacja danych (skaler dla każdej cechy wejściowej)
n_features = X.shape[2]
X_reshaped = X.reshape(-1, n_features)
scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(X_reshaped).reshape(X.shape)

# 3️⃣ Normalizacja y
y_reshaped = y.reshape(-1, 1)
scaler_y = MinMaxScaler(feature_range=(0, 1))
y_scaled = scaler_y.fit_transform(y_reshaped).reshape(y.shape)

# 4️⃣ Architektura LSTM
n_input = X.shape[1]   # np. 14 dni
n_output = y.shape[1]  # 7 dni

model = Sequential()
model.add(LSTM(128, activation='tanh', input_shape=(n_input, n_features)))
model.add(Dropout(0.2))
model.add(Dense(n_output))  # 7 dni wyjścia (tavg)
model.add(Reshape((n_output, 1)))  # shape (7,1)

# 5️⃣ Kompilacja
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

# 6️⃣ Callbacks: zapis najlepszych wag + EarlyStopping
checkpoint = ModelCheckpoint("lstm_weather_best_all_features.h5", monitor='val_loss', save_best_only=True, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

# 7️⃣ Trening
history = model.fit(
    X_scaled, y_scaled,
    epochs=50,
    batch_size=64,
    validation_split=0.1,
    callbacks=[checkpoint, early_stop]
)

# 8️⃣ Zapis finalnego modelu
model.save("lstm_weather_final_all_features.h5")
print("✅ Model LSTM zapisany jako lstm_weather_final_all_features.h5")


C:\Users\gduls\PycharmProjects\pythonProject27\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 128)            │        69,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 7)              │           903 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_2 (Reshape)             │ (None, 7, 1)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 70,023 (273.53 KB)

 Trainable params: 70,023 (273.53 KB)

...

3101/3101 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - loss: 0.0025 - mae: 0.0377 - val_loss: 0.0027 - val_mae: 0.0399
Epoch 38/50
3095/3101 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0025 - mae: 0.0376
Epoch 38: val_loss improved from 0.00274 to 0.00273, saving model to lstm_weather_best_all_features.h5


3101/3101 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - loss: 0.0025 - mae: 0.0377 - val_loss: 0.0027 - val_mae: 0.0399
Epoch 39/50
3096/3101 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0025 - mae: 0.0376
Epoch 39: val_loss did not improve from 0.00273
3101/3101 ━━━━━━━━━━━━━━━━━━━━ 25s 8ms/step - loss: 0.0024 - mae: 0.0376 - val_loss: 0.0027 - val_mae: 0.0401
Epoch 40/50
3093/3101 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0025 - mae: 0.0376
Epoch 40: val_loss did not improve from 0.00273
3101/3101 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - loss: 0.0024 - mae: 0.0376 - val_loss: 0.0028 - val_mae: 0.0402
Epoch 41/50
3097/3101 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0024 - mae: 0.0375
Epoch 41: val_loss did not improve from 0.00273
3101/3101 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - loss: 0.0024 - mae: 0.0375 - val_loss: 0.0028 - val_mae: 0.0401
Epoch 42/50
3097/3101 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0024 - mae: 0.0375
Epoch 42: val_loss did not improve from 0.00273
3101/3101 ━━━━━━━━━━━━━━━━━━━━ 2

3101/3101 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - loss: 0.0024 - mae: 0.0374 - val_loss: 0.0027 - val_mae: 0.0398
Epoch 46/50
3094/3101 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0024 - mae: 0.0373
Epoch 46: val_loss improved from 0.00272 to 0.00269, saving model to lstm_weather_best_all_features.h5


3101/3101 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - loss: 0.0024 - mae: 0.0373 - val_loss: 0.0027 - val_mae: 0.0396
Epoch 47/50
3101/3101 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0024 - mae: 0.0373
Epoch 47: val_loss did not improve from 0.00269
3101/3101 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - loss: 0.0024 - mae: 0.0374 - val_loss: 0.0028 - val_mae: 0.0401
Epoch 48/50
3095/3101 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0024 - mae: 0.0373
Epoch 48: val_loss did not improve from 0.00269
3101/3101 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - loss: 0.0024 - mae: 0.0373 - val_loss: 0.0028 - val_mae: 0.0400
Epoch 49/50
3096/3101 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0024 - mae: 0.0373
Epoch 49: val_loss did not improve from 0.00269
3101/3101 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - loss: 0.0024 - mae: 0.0373 - val_loss: 0.0027 - val_mae: 0.0397
Epoch 50/50
3096/3101 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0024 - mae: 0.0373
Epoch 50: val_loss did not improve from 0.00269
3101/3101 ━━━━━━━━━━━━━━━━━━━━ 2

✅ Model LSTM zapisany jako lstm_weather_final_all_features.h5


In [ ]:
d